In [1]:
import jax
import jax.numpy as jnp
import numpy as np
from jax.experimental.ode import odeint
import matplotlib.pyplot as plt
from functools import partial # reduces arguments to function by making some subset implicit

#from jax.experimental import stax
from jax.example_libraries import stax
from jax.example_libraries import optimizers
#from jax.experimental import optimizers

# visualization
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from moviepy.editor import ImageSequenceClip
from functools import partial
import proglog
from PIL import Image

def lagrangian(q, q_dot, m1, m2, l1, l2, g):
    t1, t2 = q     # theta 1 and theta 2
    w1, w2 = q_dot # omega 1 and omega 2

      # kinetic energy (T)
    T1 = 0.5 * m1 * (l1 * w1)**2
    T2 = 0.5 * m2 * ((l1 * w1)**2 + (l2 * w2)**2 +
                        2 * l1 * l2 * w1 * w2 * jnp.cos(t1 - t2))
    T = T1 + T2

      # potential energy (V)
    y1 = -l1 * jnp.cos(t1)
    y2 = y1 - l2 * jnp.cos(t2)
    V = m1 * g * y1 + m2 * g * y2

    return T - V

def f_analytical(state, t=0, m1=1, m2=1, l1=1, l2=1, g=9.8):
    t1, t2, w1, w2 = state
    a1 = (l2 / l1) * (m2 / (m1 + m2)) * jnp.cos(t1 - t2)
    a2 = (l1 / l2) * jnp.cos(t1 - t2)
    f1 = -(l2 / l1) * (m2 / (m1 + m2)) * (w2**2) * jnp.sin(t1 - t2) - \
    (g / l1) * jnp.sin(t1)
    f2 = (l1 / l2) * (w1**2) * jnp.sin(t1 - t2) - (g / l2) * jnp.sin(t2)
    g1 = (f1 - a1 * f2) / (1 - a1 * a2)
    g2 = (f2 - a2 * f1) / (1 - a1 * a2)
    return jnp.stack([w1, w2, g1, g2])

def equation_of_motion(lagrangian, state, t=None):
    q, q_t = jnp.split(state, 2)
    q_tt = (jnp.linalg.pinv(jax.hessian(lagrangian, 1)(q, q_t))
          @ (jax.grad(lagrangian, 0)(q, q_t)
             - jax.jacobian(jax.jacobian(lagrangian, 1), 0)(q, q_t) @ q_t))
    return jnp.concatenate([q_t, q_tt])

def solve_lagrangian(lagrangian, initial_state, **kwargs):
  # We currently run odeint on CPUs only, because its cost is dominated by
  # control flow, which is slow on GPUs.
    @partial(jax.jit, backend='cpu')
    def f(initial_state):
        return odeint(partial(equation_of_motion, lagrangian),
                  initial_state, **kwargs)
    return f(initial_state)

# Double pendulum dynamics via the rewritten Euler-Lagrange
@partial(jax.jit, backend='cpu')
def solve_autograd(initial_state, times, m1=1, m2=1, l1=1, l2=1, g=9.8):
    L = partial(lagrangian, m1=m1, m2=m2, l1=l1, l2=l2, g=g)
    return solve_lagrangian(L, initial_state, t=times, rtol=1e-10, atol=1e-10)

# Double pendulum dynamics via analytical forces taken from Diego's blog
@partial(jax.jit, backend='cpu')
def solve_analytical(initial_state, times):
    return odeint(f_analytical, initial_state, t=times, rtol=1e-10, atol=1e-10)

def normalize_dp(state):
  # wrap generalized coordinates to [-pi, pi]
  return jnp.concatenate([(state[:2] + np.pi) % (2 * np.pi) - np.pi, state[2:]])

def rk4_step(f, x, t, h):
  # one step of runge-kutta integration
    k1 = h * f(x, t)
    k2 = h * f(x + k1/2, t + h/2)
    k3 = h * f(x + k2/2, t + h/2)
    k4 = h * f(x + k3, t + h)
    return x + 1/6 * (k1 + 2 * k2 + 2 * k3 + k4)

In [2]:
time_step = 0.01
N = 1500
analytical_step = jax.jit(jax.vmap(partial(rk4_step, f_analytical, t=0.0, h=time_step)))

# x0 = np.array([-0.3*np.pi, 0.2*np.pi, 0.35*np.pi, 0.5*np.pi], dtype=np.float32)
x0 = np.array([3*np.pi/7, 3*np.pi/4, 0, 0], dtype=np.float32)
t = np.arange(N, dtype=np.float32) # time steps 0 to N
%time x_train = jax.device_get(solve_analytical(x0, t)) # dynamics for first N time steps
%time xt_train = jax.device_get(jax.vmap(f_analytical)(x_train)) # time derivatives of each state
%time y_train = jax.device_get(analytical_step(x_train)) # analytical next step

Wall time: 15.2 s
Wall time: 562 ms
Wall time: 420 ms


In [3]:
x1_new, y1_new, x2_new, y2_new = x_train[:4]

In [4]:
x1_new

array([1.3463968, 2.3561945, 0.       , 0.       ], dtype=float32)

In [5]:
timeframe = 20 


# Initialising two lists (X and Y) to store target positions (Y) and input sequences (X)
X, Y = [], []


# Looping through positions
# We will stop 'timeframe' steps before the end to avoid going out of bounds

for i in range(len(x1_new) - timeframe):
    
    # Stack the next 'timeframe' positions of both masses to serve as the input sequence
    X.append(np.vstack((x1_new[i:i+timeframe], y1_new[i:i+timeframe], x2_new[i:i+timeframe], y2_new[i:i+timeframe])).T)
    
    # Record the positions of both masses 20 steps ahead from the current timestep to set as the prediction target
    Y.append([x1_new[i+timeframe], y1_new[i+timeframe], x2_new[i+timeframe], y2_new[i+timeframe]])


# Converting position sequences to numpy array
X = np.array(X)

# Converting targets (positions 20 steps ahead) to numpy array
Y = np.array(Y)

In [6]:
X

array([], dtype=float64)

In [7]:
def rhs(t, z, L1, L2, m1, m2, g):
    """
    Returns the right-hand side of the ordinary differential equation describing the double pendulem
    """
    theta1, w1, theta2, w2 = z
    cos12 = np.cos(theta1 - theta2)
    sin12 = np.sin(theta1 - theta2)
    sin1 = np.sin(theta1)
    sin2 = np.sin(theta2)
    xi = cos12**2*m2 - m1 - m2
    w1dot = ( L1*m2*cos12*sin12*w1**2 + L2*m2*sin12*w2**2
            - m2*g*cos12*sin2      + (m1 + m2)*g*sin1)/(L1*xi)
    w2dot = -( L2*m2*cos12*sin12*w2**2 + L1*(m1 + m2)*sin12*w1**2
            + (m1 + m2)*g*sin1*cos12  - (m1 + m2)*g*sin2 )/(L2*xi)
    return w1, w1dot, w2, w2dot

In [9]:
from scipy.integrate import solve_ivp

L1, L2 = 1., 1.
m1, m2 = 3., 1.
g = 9.81     # [m/s^2]. Gravitational acceleration

z0=[np.pi/2,0,np.pi/2,0]
#z0=[0.1,0,0.1,0]
tmax, dt = 50, 0.1
t = np.arange(0, tmax+dt, dt)

ret = solve_ivp(rhs, (0,tmax), z0, t_eval=t, args=(L1, L2, m1, m2, g))
z=ret.y
print(np.shape(z))

# Extract result
theta1, w1, theta2, w2 = z[0], z[1], z[2], z[3]

(4, 501)


In [26]:
z

array([[ 1.57079633e+00,  1.52175814e+00,  1.37533880e+00, ...,
        -1.88738660e+00, -1.89872099e+00, -1.81792171e+00],
       [ 0.00000000e+00, -9.80293315e-01, -1.93995822e+00, ...,
        -5.71079056e-01,  3.44136254e-01,  1.27746850e+00],
       [ 1.57079633e+00,  1.57076483e+00,  1.56881720e+00, ...,
         2.02290547e+01,  2.01984794e+01,  2.01640628e+01],
       [ 0.00000000e+00, -1.88443214e-03, -5.87366067e-02, ...,
        -2.79896835e-01, -3.30726918e-01, -3.50010898e-01]])

In [28]:
z[:2]

array([[ 1.57079633,  1.52175814,  1.3753388 , ..., -1.8873866 ,
        -1.89872099, -1.81792171],
       [ 0.        , -0.98029331, -1.93995822, ..., -0.57107906,
         0.34413625,  1.2774685 ]])

In [11]:
z[0]

array([ 1.57079633e+00,  1.52175814e+00,  1.37533880e+00,  1.13736278e+00,
        8.25384158e-01,  4.71333510e-01,  1.24444964e-01, -1.58412895e-01,
       -3.60366960e-01, -5.42913938e-01, -7.52408788e-01, -9.66304914e-01,
       -1.14237749e+00, -1.25393409e+00, -1.28750022e+00, -1.23658828e+00,
       -1.09728260e+00, -8.64030226e-01, -5.37709505e-01, -1.40135120e-01,
        2.79784592e-01,  6.62988923e-01,  9.66201559e-01,  1.17292324e+00,
        1.28274274e+00,  1.30015286e+00,  1.22737699e+00,  1.07025696e+00,
        8.43256285e-01,  5.78394410e-01,  3.32962580e-01,  1.75128397e-01,
        8.62908931e-02, -8.86062011e-02, -3.74927190e-01, -6.97738538e-01,
       -9.90581789e-01, -1.21597229e+00, -1.35749191e+00, -1.40725853e+00,
       -1.36308217e+00, -1.22740503e+00, -1.00821179e+00, -7.24737034e-01,
       -4.15420926e-01, -1.43337980e-01,  3.22904611e-02,  1.59815184e-01,
        3.61769130e-01,  6.30928885e-01,  9.02525266e-01,  1.12645386e+00,
        1.27553088e+00,  

In [12]:
x1_angle=x_train[:,0]
x2_angle=x_train[:,1]
x1_velocity=x_train[:,2]
x2_velocity=x_train[:,3]

In [15]:
# Setting up data to forecast the position of the pendulum after 20 steps
# We'll base our prediction on the following 20 swings of the pendulum


# Timesteps ahead for our prediction
timeframe = 20 


# Initialising two lists (X and Y) to store target positions (Y) and input sequences (X)
X, Y = [], []


# Looping through positions
# We will stop 'timeframe' steps before the end to avoid going out of bounds

for i in range(len(x1_angle) - timeframe):
    
    # Stack the next 'timeframe' positions of both masses to serve as the input sequence
    X.append(np.vstack((x1_angle[i:i+timeframe], x2_angle[i:i+timeframe], x1_velocity[i:i+timeframe], x2_velocity[i:i+timeframe])).T)
    
    # Record the positions of both masses 20 steps ahead from the current timestep to set as the prediction target
    Y.append([x1_angle[i+timeframe], x2_angle[i+timeframe], x1_velocity[i+timeframe], x2_velocity[i+timeframe]])


# Converting position sequences to numpy array
X = np.array(X)

# Converting targets (positions 20 steps ahead) to numpy array
Y = np.array(Y)

In [21]:
np.shape(Y)

(1480, 4)

In [22]:
np.shape(x1_angle)

(1500,)

In [23]:
np.shape(X)

(1480, 20, 4)

In [24]:
from tensorflow.keras.models import Sequential # Importing the sequential model class
from tensorflow.keras.layers import LSTM, Dense # Importing LSTM and Dense layer classes


# ------ Defining an RNN (Recurrent Neural Network) model with LSTM (Long Short-Term Memory) layers --------

model = Sequential([
    
    LSTM(64, input_shape=(timeframe, 4), return_sequences=True),  # First LSTM layer with 64 units, returning sequences
    LSTM(32),  # Second LSTM layer with 32 units
    Dense(4)   # Output layer for future x1, y1, x2, y2 positions

])

# Compiling model with MSE loss function and Adam optimizer
model.compile(optimizer='adam', loss='mse')

In [25]:
# Training the model using input sequences to predict future positions, validating on 20% of the data
model.fit(X, Y, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
37/37 [==============================] - 9s 79ms/step - loss: 3401.6416 - val_loss: 145.2436
Epoch 2/50
37/37 [==============================] - 1s 34ms/step - loss: 3148.7976 - val_loss: 123.3779
Epoch 3/50
37/37 [==============================] - 1s 36ms/step - loss: 2998.0999 - val_loss: 102.5976
Epoch 4/50
37/37 [==============================] - 1s 36ms/step - loss: 2904.2407 - val_loss: 86.7976
Epoch 5/50
37/37 [==============================] - 1s 37ms/step - loss: 2832.2727 - val_loss: 75.9656
Epoch 6/50
37/37 [==============================] - 1s 39ms/step - loss: 2766.7549 - val_loss: 67.1969
Epoch 7/50
37/37 [==============================] - 1s 36ms/step - loss: 2704.6633 - val_loss: 60.3785
Epoch 8/50
37/37 [==============================] - 1s 36ms/step - loss: 2645.5171 - val_loss: 53.7212
Epoch 9/50
37/37 [==============================] - 1s 37ms/step - loss: 2587.7820 - val_loss: 48.4263
Epoch 10/50
37/37 [==============================] - 1s 34ms/step - lo

In [ ]:
initial_conditions = [
    [np.pi/4, np.pi/4, 0, 0],
    [np.pi/6, np.pi/3, 0, 0],
    [np.pi/3, np.pi/6, 0, 0]
]

# Defining a figure and its subplots
#fig, axes = plt.subplots(len(initial_conditions), 2, figsize=(12, 6*len(initial_conditions))) 

# Setting the main title for the figure
#fig.suptitle('Comparing Network Prediction and solve_ivp Solution', y=1.02, fontsize=15)  

# Looping through each set of initial conditions
for i, z0_var in enumerate(initial_conditions):
    
    # Computing the trajectory / motion of the double pendulum for the specified initial conditions
    ret_var = solve_ivp(rhs, (0, tmax), z0_var, t_eval=t, args=(L1, L2, m1, m2, g))
    
    # Converting angular motion data to Cartesian coordinates
    positions_var = to_cartesian(*ret_var.y, L1, L2)
    
    # Extracting the cartesian coordinates of both pendulum masses for the simulated trajectory / motion
    x1_var, y1_var, x2_var, y2_var = positions_var[:4]

    # Preparing the data sequence for the network's prediction
    X_var = [np.vstack((x1_var[j:j+timeframe], y1_var[j:j+timeframe], x2_var[j:j+timeframe], y2_var[j:j+timeframe])).T for j in range(len(x1_var) - timeframe)]
    
    X_var = np.array(X_var)

    # Using the trained RNN model to predict the future positions 
    # (More simply we are generating predictions using the trained model)

    predictions = model.predict(X_var)